In [3]:
from cs336_systems.benchmarking import benchmarking_script

In [5]:

    
benchmarking_script(
        num_layers=6,
        d_model=512,
        num_heads=8,
        d_ff=2048,
        context_length=1024,
        vocab_size=32000,
        batch_size=4,
        seq_len=256,
        device="cuda" if __import__("torch").cuda.is_available() else "cpu",
        warmup_steps=5,
        num_steps=20,
        rope_theta=10000.0
    )

{'forward_time_total': 0.2829209230003471,
 'forward_time_per_step': 0.014146046150017355,
 'forward_backward_time_total': 0.9212588479999795,
 'forward_backward_time_per_step': 0.046062942399998974,
 'backward_time_per_step': 0.03191689624998162,
 'steps': 20,
 'warmup_steps': 5,
 'batch_size': 4,
 'seq_len': 256,
 'device': 'cuda',
 'model_params': 41556480}

In [6]:
benchmarking_script(
        num_layers=6,
        d_model=512,
        num_heads=8,
        d_ff=2048,
        context_length=1024,
        vocab_size=32000,
        batch_size=4,
        seq_len=256,
        device="cuda" if __import__("torch").cuda.is_available() else "cpu",
        warmup_steps=0,
        num_steps=20,
        rope_theta=10000.0
    )

{'forward_time_total': 0.5235735200003546,
 'forward_time_per_step': 0.02617867600001773,
 'forward_backward_time_total': 0.9545965919996888,
 'forward_backward_time_per_step': 0.04772982959998444,
 'backward_time_per_step': 0.021551153599966708,
 'steps': 20,
 'warmup_steps': 0,
 'batch_size': 4,
 'seq_len': 256,
 'device': 'cuda',
 'model_params': 41556480}

In [ ]:
benchmarking_script(
        num_layers=6,
        d_model=512,
        num_heads=8,
        d_ff=2048,
        context_length=1024,
        vocab_size=32000,
        batch_size=4,
        device="cuda" if __import__("torch").cuda.is_available() else "cpu",
        warmup_steps=2,
        num_steps=20,
        rope_theta=10000.0)

{'forward_time_total': 0.3061218689999805,
 'forward_time_per_step': 0.015306093449999026,
 'forward_backward_time_total': 0.9062522469998839,
 'forward_backward_time_per_step': 0.045312612349994195,
 'backward_time_per_step': 0.03000651889999517,
 'steps': 20,
 'warmup_steps': 2,
 'batch_size': 4,
 'seq_len': 256,
 'device': 'cuda',
 'model_params': 41556480}

In [ ]:
from torch import nn
class ToyModel(nn.Module):
    def __init__(self, in_features: int, out_features: int):
        super().__init__()
        self.fc1 = nn.Linear(in_features, 10, bias=False)
        self.ln = nn.LayerNorm(10)
        self.fc2 = nn.Linear(10, out_features, bias=False)
        self.relu = nn.ReLU()

    def forward(self, x):
     x = self.relu(self.fc1(x))
     x = self.ln(x)
     x = self.fc2(x)
     return x

In [36]:
import torch
x = torch.randn(4, 10)  
toy_model = ToyModel(10, 10)

output = toy_model(x)
[param.dtype for param in toy_model.parameters()]


[torch.float32, torch.float32, torch.float32, torch.float32]

In [38]:
# Demonstrating autocast behavior with FP16 mixed precision
print("Original model parameters (should be float32):")
print([param.dtype for param in toy_model.parameters()])

# Move model and input to CUDA for autocast
toy_model = toy_model.cuda()
x = x.cuda()

print("\nUsing autocast for mixed precision:")
with torch.autocast(device_type='cuda', dtype=torch.float16):
    y = toy_model(x)
    print("Model parameters during autocast (unchanged):")
    print([param.dtype for param in toy_model.parameters()])
    print(f"Output tensor dtype: {y.dtype}")
    print(f"First feed-forward layer output dtype during autocast:")
    with torch.no_grad():
        x_fc1 = toy_model.fc1(x)
        print(f"  fc1 output: {x_fc1.dtype}")
        x_relu = toy_model.relu(x_fc1)
        print(f"  relu output: {x_relu.dtype}")
        x_ln = toy_model.ln(x_relu)
        print(f"  layer norm output: {x_ln.dtype}")

print("\nPredicted logits analysis:")
print(f"Output shape: {y.shape}")
print(f"Output range: [{y.min().item():.4f}, {y.max().item():.4f}]")

print("\nLoss computation:")
loss = y.sum()
print(f"Loss dtype: {loss.dtype}")
print(f"Loss value: {loss.item():.4f}")

print("\nGradients analysis:")
loss.backward()
print("Gradient dtypes:")
for name, param in toy_model.named_parameters():
    if param.grad is not None:
        print(f"  {name}: param={param.dtype}, grad={param.grad.dtype}")

Original model parameters (should be float32):
[torch.float32, torch.float32, torch.float32, torch.float32]

Using autocast for mixed precision:
Model parameters during autocast (unchanged):
[torch.float32, torch.float32, torch.float32, torch.float32]
Output tensor dtype: torch.float16
First feed-forward layer output dtype during autocast:
  fc1 output: torch.float16
  relu output: torch.float16
  layer norm output: torch.float32

Predicted logits analysis:
Output shape: torch.Size([4, 10])
Output range: [-1.0049, 1.1182]

Loss computation:
Loss dtype: torch.float16
Loss value: -4.7617

Gradients analysis:
Gradient dtypes:
  fc1.weight: param=torch.float32, grad=torch.float32
  ln.weight: param=torch.float32, grad=torch.float32
  ln.bias: param=torch.float32, grad=torch.float32
  fc2.weight: param=torch.float32, grad=torch.float32


In [39]:
import time
import torch
s = torch.tensor(0,dtype=torch.float16)
for i in range(100000):
    s += torch.tensor(0.01,dtype=torch.float16)
end_time = time.time()
print(f"Result: {s}")

Result: 32.0
